In [0]:
# Import required PySpark libraries for data processing
from pyspark.sql import * # For Spark SQL functions like col, trim, filter, etc.
from pyspark.sql.window import Window# For window functions to handle duplicates
from pyspark.sql.functions import *    

# Read the data from Bronze layer
customers_df = spark.read.format("delta").load("jaffle_shop_retail/bronze/raw_customers")
orders_df = spark.read.format("delta").load("jaffle_shop_retail/bronze/raw_orders")
order_items_df = spark.read.format("delta").load("jaffle_shop_retail/bronze/raw_order_items")
products_df = spark.read.format("delta").load("jaffle_shop_retail/bronze/raw_products")
supplies_df = spark.read.format("delta").load("jaffle_shop_retail/bronze/raw_supplies")
stores_df = spark.read.format("delta").load("jaffle_shop_retail/bronze/raw_stores")

In [0]:
# Load customers_bronze from Unity Catalog
customers_bronze = spark.table("jaffle_shop_retail.bronze.raw_customers")

from pyspark.sql.types import StringType
from pyspark.sql.functions import col, when, lit, sha2, current_timestamp

# Step 1: Select and cast columns to proper data types
customers_selected = customers_bronze.select(
    col("id").cast(StringType()).alias("customer_id"),
    col("name").cast(StringType()).alias("customer_name")
)

# Step 2: Filter out null values from critical columns
customers_no_nulls = customers_selected.filter(
    col("customer_id").isNotNull() &
    col("customer_name").isNotNull()
)

# Step 3: Handle missing customer names
customers_with_name = customers_no_nulls.withColumn(
    "customer_name",
    when(col("customer_name").isNull(), lit("Unknown Customer"))
    .otherwise(col("customer_name"))
)

# Step 4: Remove duplicate customers
customers_deduped = customers_with_name.dropDuplicates(["customer_id"])

# Step 5: Add customer_key hash column
customers_with_key = customers_deduped.withColumn("customer_key", sha2(col("customer_id"), 256))

# Step 6: Add loaded_at timestamp
customers_silver_final = customers_with_key.withColumn("loaded_at", current_timestamp())

# Step 7: Write to Silver schema
customers_silver_final.write.format("delta").mode("overwrite").saveAsTable("jaffle_shop_retail.silver.customers")

# Step 8: Display final result
display(customers_silver_final)

In [0]:
# Load orders_bronze from Unity Catalog
orders_bronze = spark.table("jaffle_shop_retail.bronze.raw_orders")

from pyspark.sql.types import StringType, TimestampType, DoubleType
from pyspark.sql.functions import col, when, lit, sha2, current_timestamp

# Step 1: Select and cast columns to proper data types
orders_selected = orders_bronze.select(
    col("id").cast(StringType()).alias("order_id"),
    col("customer").cast(StringType()).alias("customer_id"),
    col("ordered_at").cast(TimestampType()).alias("ordered_at"),
    col("store_id").cast(StringType()).alias("store_id"),
    col("subtotal").cast(DoubleType()).alias("subtotal"),
    col("tax_paid").cast(DoubleType()).alias("tax_paid"),
    col("order_total").cast(DoubleType()).alias("order_total")
)

# Step 2: Filter out null values from critical columns
orders_no_nulls = orders_selected.filter(
    col("order_id").isNotNull() &
    col("customer_id").isNotNull() &
    col("ordered_at").isNotNull() &
    col("store_id").isNotNull()
)

# Step 3: Handle missing financial values
orders_with_financials = orders_no_nulls.withColumn(
    "subtotal",
    when(col("subtotal").isNull(), lit(0.0)).otherwise(col("subtotal"))
).withColumn(
    "tax_paid",
    when(col("tax_paid").isNull(), lit(0.0)).otherwise(col("tax_paid"))
).withColumn(
    "order_total",
    when(col("order_total").isNull(), lit(0.0)).otherwise(col("order_total"))
)

# Step 4: Fix negative financial values
orders_valid_financials = orders_with_financials.withColumn(
    "subtotal",
    when(col("subtotal") < 0, lit(0.0)).otherwise(col("subtotal"))
).withColumn(
    "tax_paid",
    when(col("tax_paid") < 0, lit(0.0)).otherwise(col("tax_paid"))
).withColumn(
    "order_total",
    when(col("order_total") < 0, lit(0.0)).otherwise(col("order_total"))
)

# Step 5: Remove duplicate orders
orders_deduped = orders_valid_financials.dropDuplicates(["order_id"])

# Step 6: Add order_key hash column
orders_with_key = orders_deduped.withColumn("order_key", sha2(col("order_id"), 256))

# Step 7: Add loaded_at timestamp
orders_silver_final = orders_with_key.withColumn("loaded_at", current_timestamp())

# Step 8: Write to Silver schema
orders_silver_final.write.format("delta").mode("overwrite").saveAsTable("jaffle_shop_retail.silver.orders")

# Step 9: Display final result
display(orders_silver_final)

In [0]:
# Load order_items_bronze from Unity Catalog
order_items_bronze = spark.table("jaffle_shop_retail.bronze.raw_order_items")

from pyspark.sql.types import StringType
from pyspark.sql.functions import col, when, lit, sha2, concat, current_timestamp

# Step 1: Select and cast columns to proper data types
order_items_selected = order_items_bronze.select(
    col("id").cast(StringType()).alias("order_item_id"),
    col("order_id").cast(StringType()).alias("order_id"),
    col("sku").cast(StringType()).alias("product_sku")
)

# Step 2: Filter out null values from critical columns
order_items_no_nulls = order_items_selected.filter(
    col("order_item_id").isNotNull() &
    col("order_id").isNotNull() &
    col("product_sku").isNotNull()
)

# Step 3: Handle missing SKUs
order_items_with_sku = order_items_no_nulls.withColumn(
    "product_sku",
    when(col("product_sku").isNull(), lit("UNKNOWN"))
    .otherwise(col("product_sku"))
)

# Step 4: Remove duplicate order items
order_items_deduped = order_items_with_sku.dropDuplicates(["order_item_id"])

# Step 5: Add order_item_key hash column
order_items_with_key = order_items_deduped.withColumn(
    "order_item_key", 
    sha2(concat(col("order_item_id"), col("order_id"), col("product_sku")), 256)
)

# Step 6: Add loaded_at timestamp
order_items_silver_final = order_items_with_key.withColumn("loaded_at", current_timestamp())

# Step 7: Write to Silver schema
order_items_silver_final.write.format("delta").mode("overwrite").saveAsTable("jaffle_shop_retail.silver.order_items")

# Step 8: Display final result
display(order_items_silver_final)

In [0]:
# Load products_bronze from Unity Catalog
products_bronze = spark.table("jaffle_shop_retail.bronze.raw_products")

from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import col, when, lit, sha2, current_timestamp

# Step 1: Select and cast columns to proper data types
products_selected = products_bronze.select(
    col("sku").cast(StringType()).alias("sku"),
    col("name").cast(StringType()).alias("product_name"),
    col("type").cast(StringType()).alias("product_type"),
    col("price").cast(IntegerType()).alias("price"),
    col("description").cast(StringType()).alias("description")
)

# Step 2: Filter out null values from critical columns
products_no_nulls = products_selected.filter(
    col("sku").isNotNull() &
    col("product_name").isNotNull() &
    col("product_type").isNotNull() &
    col("price").isNotNull()
)

# Step 3: Handle missing descriptions
products_with_description = products_no_nulls.withColumn(
    "description",
    when(col("description").isNull(), lit("No description available"))
    .otherwise(col("description"))
)

# Step 4: Handle missing product types
products_with_type = products_with_description.withColumn(
    "product_type",
    when(col("product_type").isNull(), lit("unknown"))
    .otherwise(col("product_type"))
)

# Step 5: Filter out invalid prices
products_valid_price = products_with_type.filter(col("price") > 0)

# Step 6: Remove duplicate SKUs
products_deduped = products_valid_price.dropDuplicates(["sku"])

# Step 7: Add product_id hash column
products_with_id = products_deduped.withColumn("product_id", sha2(col("sku"), 256))

# Step 8: Add loaded_at timestamp
products_silver_final = products_with_id.withColumn("loaded_at", current_timestamp())

# Step 9: Write to Silver schema
products_silver_final.write.format("delta").mode("overwrite").saveAsTable("jaffle_shop_retail.silver.products")

# Step 10: Display final result
display(products_silver_final)

In [0]:
# Load stores_bronze from Unity Catalog
stores_bronze = spark.table("jaffle_shop_retail.bronze.raw_stores")

from pyspark.sql.types import StringType, TimestampType, DoubleType
from pyspark.sql.functions import col, when, lit, sha2, current_timestamp

# Step 1: Select and cast columns to proper data types
stores_selected = stores_bronze.select(
    col("id").cast(StringType()).alias("store_id"),
    col("name").cast(StringType()).alias("store_name"),
    col("opened_at").cast(TimestampType()).alias("opened_at"),
    col("tax_rate").cast(DoubleType()).alias("tax_rate")
)

# Step 2: Filter out null values from critical columns
stores_no_nulls = stores_selected.filter(
    col("store_id").isNotNull() &
    col("store_name").isNotNull() &
    col("opened_at").isNotNull()
)

# Step 3: Handle missing tax rates
stores_with_tax_rate = stores_no_nulls.withColumn(
    "tax_rate",
    when(col("tax_rate").isNull(), lit(0.0))
    .otherwise(col("tax_rate"))
)

# Step 4: Fix negative tax rates
stores_valid_tax = stores_with_tax_rate.withColumn(
    "tax_rate",
    when(col("tax_rate") < 0, lit(0.0))
    .otherwise(col("tax_rate"))
)

# Step 5: Normalize tax rates (convert percentages > 1 to decimal)
stores_normalized_tax = stores_valid_tax.withColumn(
    "tax_rate",
    when(col("tax_rate") > 1, col("tax_rate") / 100)
    .otherwise(col("tax_rate"))
)

# Step 6: Remove duplicate stores
stores_deduped = stores_normalized_tax.dropDuplicates(["store_id"])

# Step 7: Add store_key hash column
stores_with_key = stores_deduped.withColumn("store_key", sha2(col("store_id"), 256))

# Step 8: Add loaded_at timestamp
stores_silver_final = stores_with_key.withColumn("loaded_at", current_timestamp())

# Step 9: Write to Silver schema
stores_silver_final.write.format("delta").mode("overwrite").saveAsTable("jaffle_shop_retail.silver.stores")

# Step 10: Display final result
display(stores_silver_final)

In [0]:
# Load supplies_bronze from Unity Catalog
supplies_bronze = spark.table("jaffle_shop_retail.bronze.raw_supplies")

from pyspark.sql.types import StringType, IntegerType, BooleanType
from pyspark.sql.functions import col, when, lit, sha2, concat, current_timestamp

# Step 1: Select and cast columns to proper data types
supplies_selected = supplies_bronze.select(
    col("id").cast(StringType()).alias("supply_id"),
    col("name").cast(StringType()).alias("supply_name"),
    col("cost").cast(IntegerType()).alias("cost"),
    col("perishable").cast(BooleanType()).alias("perishable"),
    col("sku").cast(StringType()).alias("product_sku")
)

# Step 2: Filter out null values from critical columns
supplies_no_nulls = supplies_selected.filter(
    col("supply_id").isNotNull() &
    col("supply_name").isNotNull() &
    col("cost").isNotNull() &
    col("product_sku").isNotNull()
)

# Step 3: Handle missing perishable flags
supplies_with_perishable = supplies_no_nulls.withColumn(
    "perishable",
    when(col("perishable").isNull(), lit(False))
    .otherwise(col("perishable"))
)

# Step 4: Fix negative costs
supplies_valid_cost = supplies_with_perishable.withColumn(
    "cost",
    when(col("cost") < 0, lit(0))
    .otherwise(col("cost"))
)

# Step 5: Remove duplicate supply-SKU combinations
supplies_deduped = supplies_valid_cost.dropDuplicates(["supply_id", "product_sku"])

# Step 6: Add supply_key hash column
supplies_with_key = supplies_deduped.withColumn(
    "supply_key", 
    sha2(concat(col("supply_id"), col("product_sku")), 256)
)

# Step 7: Add loaded_at timestamp
supplies_silver_final = supplies_with_key.withColumn("loaded_at", current_timestamp())

# Step 8: Write to Silver schema
supplies_silver_final.write.format("delta").mode("overwrite").saveAsTable("jaffle_shop_retail.silver.supplies")

# Step 9: Display final result
display(supplies_silver_final)

In [0]:
# Check all tables in silver schema
display(spark.sql("SHOW TABLES IN jaffle_shop_retail.silver"))

# Verify schemas of each table
print("Products Schema:")
display(spark.sql("DESCRIBE jaffle_shop_retail.silver.products"))

print("Stores Schema:")
display(spark.sql("DESCRIBE jaffle_shop_retail.silver.stores"))

print("Supplies Schema:")
display(spark.sql("DESCRIBE jaffle_shop_retail.silver.supplies"))

print("Customers Schema:")
display(spark.sql("DESCRIBE jaffle_shop_retail.silver.customers"))

print("Orders Schema:")
display(spark.sql("DESCRIBE jaffle_shop_retail.silver.orders"))

print("Order Items Schema:")
display(spark.sql("DESCRIBE jaffle_shop_retail.silver.order_items"))

# Count records in each silver table
print("Record counts in silver tables:")
display(spark.sql("""
    SELECT 'products' as table_name, COUNT(*) as count FROM jaffle_shop_retail.silver.products 
    UNION ALL 
    SELECT 'stores' as table_name, COUNT(*) as count FROM jaffle_shop_retail.silver.stores 
    UNION ALL 
    SELECT 'supplies' as table_name, COUNT(*) as count FROM jaffle_shop_retail.silver.supplies
    UNION ALL
    SELECT 'customers' as table_name, COUNT(*) as count FROM jaffle_shop_retail.silver.customers
    UNION ALL
    SELECT 'orders' as table_name, COUNT(*) as count FROM jaffle_shop_retail.silver.orders
    UNION ALL
    SELECT 'order_items' as table_name, COUNT(*) as count FROM jaffle_shop_retail.silver.order_items
"""))

In [0]:
# Data quality checks after loading to silver
from pyspark.sql.functions import count, when, isnan, isnull

def check_data_quality(table_name):
    df = spark.table(f"jaffle_shop_retail.silver.{table_name}")
    total_count = df.count()
    null_counts = df.select([
        count(when(isnull(c), c)).alias(c) for c in df.columns
    ])
    print(f"Data Quality Check for {table_name}:")
    print(f"Total records: {total_count}")
    print("Null counts per column:")
    display(null_counts)
    return total_count

# Run quality checks for all tables
for table in ["products", "stores", "supplies", "customers", "orders", "order_items"]:
    check_data_quality(table)


# UNIT TESTING FOR ALL 6 SILVER TABLES


In [0]:
# Import testing dependencies
import unittest
from pyspark.sql.types import *
from datetime import datetime

# Test class for Customers Silver Layer
class TestCustomersSilver(unittest.TestCase):

    def setUp(self):
        self.df = spark.table("jaffle_shop_retail.silver.customers")

    def test_customers_schema(self):
        expected_columns = ['customer_id', 'customer_name', 'customer_key', 'loaded_at']
        self.assertListEqual(self.df.columns, expected_columns)

    def test_customers_no_null_ids(self):
        null_count = self.df.filter(col("customer_id").isNull()).count()
        self.assertEqual(null_count, 0)

    def test_customers_no_duplicate_ids(self):
        duplicate_count = self.df.groupBy("customer_id").count().filter(col("count") > 1).count()
        self.assertEqual(duplicate_count, 0)

    def test_customers_valid_names(self):
        invalid_names = self.df.filter(
            (col("customer_name").isNull()) |
            (trim(col("customer_name")) == "") |
            (length(trim(col("customer_name"))) < 2)
        )
        self.assertEqual(invalid_names.count(), 0)

    def test_customers_has_records(self):
        self.assertGreater(self.df.count(), 0)

# Test class for Orders Silver Layer
class TestOrdersSilver(unittest.TestCase):

    def setUp(self):
        self.df = spark.table("jaffle_shop_retail.silver.orders")

    def test_orders_schema(self):
        expected_columns = ['order_id', 'customer_id', 'ordered_at', 'store_id',
                            'subtotal', 'tax_paid', 'order_total', 'order_key', 'loaded_at']
        self.assertListEqual(self.df.columns, expected_columns)

    def test_orders_financial_validation(self):
        invalid_financials = self.df.filter(
            (col("subtotal") < 0) |
            (col("tax_paid") < 0) |
            (col("order_total") < 0)
        )
        self.assertEqual(invalid_financials.count(), 0)

    def test_orders_no_null_critical_fields(self):
        null_count = self.df.filter(
            col("order_id").isNull() |
            col("customer_id").isNull() |
            col("ordered_at").isNull() |
            col("store_id").isNull()
        ).count()
        self.assertEqual(null_count, 0)

    def test_orders_has_records(self):
        self.assertGreater(self.df.count(), 0)

# Test class for Products Silver Layer
class TestProductsSilver(unittest.TestCase):

    def setUp(self):
        self.df = spark.table("jaffle_shop_retail.silver.products")

    def test_products_schema(self):
        expected_columns = ['sku', 'product_name', 'product_type', 'price',
                            'description', 'product_id', 'loaded_at']
        self.assertListEqual(self.df.columns, expected_columns)

    def test_products_positive_prices(self):
        non_positive_prices = self.df.filter(col("price") <= 0)
        self.assertEqual(non_positive_prices.count(), 0)

    def test_products_no_duplicate_skus(self):
        duplicate_count = self.df.groupBy("sku").count().filter(col("count") > 1).count()
        self.assertEqual(duplicate_count, 0)

    def test_products_has_records(self):
        self.assertGreater(self.df.count(), 0)

# Test class for Order Items Silver Layer
class TestOrderItemsSilver(unittest.TestCase):

    def setUp(self):
        self.df = spark.table("jaffle_shop_retail.silver.order_items")

    def test_order_items_schema(self):
        expected_columns = ['order_item_id', 'order_id', 'product_sku', 'order_item_key', 'loaded_at']
        self.assertListEqual(self.df.columns, expected_columns)

    def test_order_items_no_null_fields(self):
        null_count = self.df.filter(
            col("order_item_id").isNull() |
            col("order_id").isNull() |
            col("product_sku").isNull()
        ).count()
        self.assertEqual(null_count, 0)

    def test_order_items_has_records(self):
        self.assertGreater(self.df.count(), 0)

# Test class for Stores Silver Layer
class TestStoresSilver(unittest.TestCase):

    def setUp(self):
        self.df = spark.table("jaffle_shop_retail.silver.stores")

    def test_stores_schema(self):
        expected_columns = ['store_id', 'store_name', 'opened_at', 'tax_rate', 'store_key', 'loaded_at']
        self.assertListEqual(self.df.columns, expected_columns)

    def test_stores_valid_tax_rates(self):
        invalid_tax_rates = self.df.filter(
            (col("tax_rate") < 0) | (col("tax_rate") > 1)
        )
        self.assertEqual(invalid_tax_rates.count(), 0)

    def test_stores_no_duplicate_ids(self):
        duplicate_count = self.df.groupBy("store_id").count().filter(col("count") > 1).count()
        self.assertEqual(duplicate_count, 0)

    def test_stores_has_records(self):
        self.assertGreater(self.df.count(), 0)

# Test class for Supplies Silver Layer
class TestSuppliesSilver(unittest.TestCase):

    def setUp(self):
        self.df = spark.table("jaffle_shop_retail.silver.supplies")

    def test_supplies_schema(self):
        expected_columns = ['supply_id', 'supply_name', 'cost', 'perishable',
                            'product_sku', 'supply_key', 'loaded_at']
        self.assertListEqual(self.df.columns, expected_columns)

    def test_supplies_non_negative_cost(self):
        negative_costs = self.df.filter(col("cost") < 0)
        self.assertEqual(negative_costs.count(), 0)

    def test_supplies_no_duplicate_supply_sku(self):
        duplicate_count = self.df.groupBy("supply_id", "product_sku").count().filter(col("count") > 1).count()
        self.assertEqual(duplicate_count, 0)

    def test_supplies_has_records(self):
        self.assertGreater(self.df.count(), 0)

# Integration Tests for all 6 tables
class TestIntegrationSilver(unittest.TestCase):

    def test_orders_customers_integration(self):
        """Test that all orders have valid customer references"""
        orders_df = spark.table("jaffle_shop_retail.silver.orders")
        customers_df = spark.table("jaffle_shop_retail.silver.customers")

        invalid_orders = orders_df.join(
            customers_df,
            orders_df.customer_id == customers_df.customer_id,
            "left_anti"
        )
        self.assertEqual(invalid_orders.count(), 0, "All orders should have valid customer IDs")

    def test_order_items_orders_integration(self):
        """Test that all order items have valid order references"""
        order_items_df = spark.table("jaffle_shop_retail.silver.order_items")
        orders_df = spark.table("jaffle_shop_retail.silver.orders")

        invalid_order_items = order_items_df.join(
            orders_df,
            order_items_df.order_id == orders_df.order_id,
            "left_anti"
        )
        self.assertEqual(invalid_order_items.count(), 0, "All order items should have valid order IDs")

    def test_order_items_products_integration(self):
        """Test that all order items have valid product SKU references"""
        order_items_df = spark.table("jaffle_shop_retail.silver.order_items")
        products_df = spark.table("jaffle_shop_retail.silver.products")

        invalid_order_items = order_items_df.join(
            products_df,
            order_items_df.product_sku == products_df.sku,
            "left_anti"
        )
        # This might have some if there are new products not in products table
        print(f"Order items with invalid product SKUs: {invalid_order_items.count()}")

    def test_orders_stores_integration(self):
        """Test that all orders have valid store references"""
        orders_df = spark.table("jaffle_shop_retail.silver.orders")
        stores_df = spark.table("jaffle_shop_retail.silver.stores")

        invalid_orders = orders_df.join(
            stores_df,
            orders_df.store_id == stores_df.store_id,
            "left_anti"
        )
        self.assertEqual(invalid_orders.count(), 0, "All orders should have valid store IDs")

    def test_supplies_products_integration(self):
        """Test that all supplies have valid product SKU references"""
        supplies_df = spark.table("jaffle_shop_retail.silver.supplies")
        products_df = spark.table("jaffle_shop_retail.silver.products")

        invalid_supplies = supplies_df.join(
            products_df,
            supplies_df.product_sku == products_df.sku,
            "left_anti"
        )
        # This might have some if there are supplies for products not in products table
        print(f"Supplies with invalid product SKUs: {invalid_supplies.count()}")

# Data Quality Tests for all tables
class TestDataQualitySilver(unittest.TestCase):

    def test_all_6_tables_have_records(self):
        """Test that all 6 silver tables have records"""
        tables = ["customers", "orders", "order_items", "products", "stores", "supplies"]

        for table in tables:
            count = spark.table(f"jaffle_shop_retail.silver.{table}").count()
            self.assertGreater(count, 0, f"Table {table} should have records")
            print(f"✓ {table}: {count} records")

    def test_all_tables_have_loaded_at(self):
        """Test that all 6 tables have loaded_at timestamp"""
        tables = ["customers", "orders", "order_items", "products", "stores", "supplies"]

        for table in tables:
            df = spark.table(f"jaffle_shop_retail.silver.{table}")
            null_loaded_at = df.filter(col("loaded_at").isNull()).count()
            self.assertEqual(null_loaded_at, 0, f"Table {table} should have loaded_at timestamps")

    def test_all_tables_have_hash_keys(self):
        """Test that all tables have their respective hash keys"""
        table_key_mapping = {
            "customers": "customer_key",
            "orders": "order_key",
            "order_items": "order_item_key",
            "products": "product_id",
            "stores": "store_key",
            "supplies": "supply_key"
        }

        for table, key_column in table_key_mapping.items():
            df = spark.table(f"jaffle_shop_retail.silver.{table}")
            null_keys = df.filter(col(key_column).isNull()).count()
            self.assertEqual(null_keys, 0, f"Table {table} should have non-null {key_column}")

integration_test = unittest.TestLoader().loadTestsFromTestCase(TestIntegrationSilver)
unittest.TextTestRunner(verbosity=1).run(integration_test)

In [0]:
def run_table_tests(table_name):
    try:
        df = spark.table(table_name)
        display(
            df.selectExpr(
                f"'{table_name}' as table_name",
                "count(*) as row_count"
            )
        )
    except Exception as e:
        print(f"Table not found or error for {table_name}: {e}")

run_table_tests("jaffle_shop_retail.silver.customers")
run_table_tests("jaffle_shop_retail.silver.orders")
run_table_tests("jaffle_shop_retail.silver.order_items")
run_table_tests("jaffle_shop_retail.silver.supplies")
run_table_tests("jaffle_shop_retail.silver.stores")
run_table_tests("jaffle_shop_retail.silver.products")

